### Importing Libraries and Reading CSV Files

In [1]:
import pandas as pd

df_2019 = pd.read_csv("2019.csv")
df_2020_2021 = pd.read_csv("2020-2021.csv")
df_2022_2023 = pd.read_csv("2022-2023.csv")

### Merging  Files

In [2]:
df_all = pd.concat([df_2019, df_2020_2021, df_2022_2023], ignore_index=True)

In [3]:
print(df_all.shape)
print(df_all.head())

(112808, 16)
   Year App.                 Taxon     Class          Order         Family  \
0  2019    I  Haliaeetus albicilla      Aves  Falconiformes   Accipitridae   
1  2019    I        Harpia harpyja      Aves  Falconiformes   Accipitridae   
2  2019    I        Harpia harpyja      Aves  Falconiformes   Accipitridae   
3  2019    I        Harpia harpyja      Aves  Falconiformes   Accipitridae   
4  2019    I    Alligator sinensis  Reptilia     Crocodylia  Alligatoridae   

        Genus Importer Exporter Origin  Importer reported quantity  \
0  Haliaeetus       IS       DK     XX                         NaN   
1      Harpia       FR       CH     MO                        2.66   
2      Harpia       FR       CH     XX                         NaN   
3      Harpia       TT       SR    NaN                         NaN   
4   Alligator       CH       SI     US                         NaN   

   Exporter reported quantity                      Term                 Unit  \
0                

In [4]:
### Cleaning 

#### This step avoids confusion with actual country codes like `NA` (Namibia) by explicitly marking missing values.

In [5]:
df_2019 = pd.read_csv("2019.csv", keep_default_na=False, na_values=["", " ", "null", "NULL", "NaN", "nan", "N/A"], dtype=str)
df_2020_2021 = pd.read_csv("2020-2021.csv", keep_default_na=False, na_values=["", " ", "null", "NULL", "NaN", "nan", "N/A"], dtype=str)
df_2022_2023 = pd.read_csv("2022-2023.csv", keep_default_na=False, na_values=["", " ", "null", "NULL", "NaN", "nan", "N/A"], dtype=str)
df_all = pd.concat([df_2019, df_2020_2021, df_2022_2023], ignore_index=True)
cols = ["Origin", "Exporter", "Importer"]
for col in cols:
    df_all[col] = df_all[col].apply(lambda x: pd.NA if x == "" else x.strip() if isinstance(x, str) else x)

#### Checking Missing Values in the Dataset


In [6]:
df_all.isna().sum()

Year                              0
App.                              0
Taxon                             0
Class                             2
Order                            32
Family                         1529
Genus                          1743
Importer                          0
Exporter                          0
Origin                        66900
Importer reported quantity    56267
Exporter reported quantity    37167
Term                              0
Unit                          36068
Purpose                        1571
Source                         2151
dtype: int64

In [7]:
missing_class = df_all[df_all["Class"].isna()][["Taxon", "Class"]]
print(missing_class.head(20))
print("Total missing Class:", missing_class.shape[0])

          Taxon Class
1019   Chordata   NaN
25145  Cnidaria   NaN
Total missing Class: 2


In [8]:
import numpy as np
import pandas as pd


df_all["Importer reported quantity"] = pd.to_numeric(
    df_all["Importer reported quantity"], errors='coerce'
).fillna(0)

df_all["Exporter reported quantity"] = pd.to_numeric(
    df_all["Exporter reported quantity"], errors='coerce'
).fillna(0)

df_all["Reported_Quantity"] = np.maximum(
    df_all["Importer reported quantity"],
    df_all["Exporter reported quantity"]
)

print("Quantity column updated with CITES Gross Trade methodology")
print(f"Records updated: {len(df_all)}")

Quantity column updated with CITES Gross Trade methodology
Records updated: 112808


In [9]:
import requests

api_key = "BLQkdzEKs2ejg4WMA87uSgtt"

def get_class(taxon_name):
    url = f"https://api.speciesplus.net/v2/taxa?search={taxon_name}"
    headers = {"Authorization": f"Bearer {api_key}"}
    resp = requests.get(url, headers=headers)
    if resp.status_code == 200:
        results = resp.json().get("results", [])
        if results:
            return results[0].get("class")
    return None

taxa = ["Chordata", "Cnidaria"]

for taxon in taxa:
    taxon_class = get_class(taxon)
    print(taxon, ":", taxon_class)


Chordata : None
Cnidaria : None


**Rows where Taxon was a higher-level or ambiguous group ("Chordata" or "Cnidaria") were removed because their Class cannot be accurately determined. Verification was done using the Species+ API and the CITES checklist.**

In [10]:
taxa_to_remove = ["Chordata", "Cnidaria"]
df_all = df_all[~df_all["Taxon"].isin(taxa_to_remove)].reset_index(drop=True)


In [11]:
df_all.isna().sum()

Year                              0
App.                              0
Taxon                             0
Class                             0
Order                            30
Family                         1527
Genus                          1741
Importer                          0
Exporter                          0
Origin                        66898
Importer reported quantity        0
Exporter reported quantity        0
Term                              0
Unit                          36067
Purpose                        1570
Source                         2151
Reported_Quantity                 0
dtype: int64

In [12]:
unique_classes = df_all["Class"].dropna().unique()
print(unique_classes)


['Aves' 'Reptilia' 'Mammalia' 'Amphibia' 'Gastropoda' 'Elasmobranchii'
 'Actinopteri' 'Anthozoa' 'Hirudinoidea' 'Hydrozoa' 'Cephalopoda'
 'Insecta' 'Arachnida' 'Bivalvia' 'Holothuroidea' 'Coelacanthi'
 'Dipneusti']


In [13]:
class_dict = {
    'Aves': 'Birds',
    'Reptilia': 'Reptiles',
    'Mammalia': 'Mammals',
    'Amphibia': 'Amphibians',
    'Gastropoda': 'Gastropods',
    'Elasmobranchii': 'Sharks and Rays',
    'Actinopteri': 'Ray-finned Fishes',
    'Anthozoa': 'Corals',
    'Hirudinoidea': 'Leeches',
    'Hydrozoa': 'Hydroids',
    'Cephalopoda': 'Cephalopods',
    'Insecta': 'Insects',
    'Arachnida': 'Arachnids',
    'Bivalvia': 'Bivalves',
    'Holothuroidea': 'Sea Cucumbers',
    'Coelacanthi': 'Coelacanths',
    'Dipneusti': 'Lungfishes'
}

df_all['Class_EN'] = df_all['Class'].map(class_dict)

In [14]:
unique_origins = df_all['Origin'].unique()
print(unique_origins)
print(f"Number of unique values in Origin: {len(unique_origins)}")


['XX' 'MO' nan 'US' 'GB' 'NZ' 'CA' 'GL' 'NA' 'PK' 'ID' 'PE' 'BR' 'CO' 'CU'
 'NP' 'CN' 'MM' 'TH' 'AR' 'MG' 'MX' 'PG' 'PA' 'PR' 'TR' 'MV' 'SN' 'MD'
 'IN' 'VN' 'BN' 'MY' 'SG' 'KH' 'MZ' 'TZ' 'CD' 'ZA' 'IT' 'ZM' 'KG' 'QA'
 'ZW' 'ET' 'SL' 'CG' 'TW' 'LR' 'MU' 'GA' 'GN' 'CM' 'NG' 'GH' 'CI' 'EG'
 'GU' 'KE' 'SD' 'GP' 'DE' 'NO' 'FR' 'GY' 'PH' 'AU' 'PW' 'SB' 'CK' 'FJ'
 'JP' 'OM' 'TO' 'YE' 'YT' 'ML' 'NE' 'TG' 'BJ' 'VE' 'BO' 'HN' 'NI' 'PY'
 'EC' 'MA' 'FM' 'NL' 'CV' 'PT' 'IL' 'SO' 'BS' 'TT' 'JM' 'BB' 'SR' 'RU'
 'AZ' 'BW' 'TJ' 'MN' 'KZ' 'GG' 'CL' 'UG' 'KN' 'TD' 'MR' 'AQ' 'MW' 'IS'
 'NT' 'IR' 'SE' 'AW' 'BF' 'CF' 'RW' 'BI' 'AO' 'XV' 'BD' 'AI' 'BZ' 'CR'
 'DM' 'GT' 'MS' 'SV' 'HS' 'ES' 'NC' 'UY' 'PF' 'BY' 'KM' 'LA' 'KI' 'VC'
 'LC' 'DO' 'TC' 'CW' 'SC' 'UZ' 'LB' 'RO' 'HR' 'SS' 'GQ' 'AG' 'UA' 'AT'
 'KW' 'BT' 'BE' 'BH' 'CH' 'HK' 'GF' 'ME' 'FO' 'EE' 'VU' 'CZ' 'IM' 'RE'
 'TF' 'DJ' 'SA' 'GD' 'SJ' 'FI' 'DK' 'SZ' 'BG' 'GM' 'AE' 'KR' 'RS' 'LK'
 'SI' 'AM' 'SK' 'GR' 'JO' 'MK' 'WF' 'WS' 'SH' 'GS' 'TN' 'SY' 'DZ' 'CY'
 '

In [15]:
country_dict = {
"IS": "Iceland", "DK": "Denmark", "FR": "France", "CH": "Switzerland", "TT": "Trinidad and Tobago",
    "SR": "Suriname", "SI": "Slovenia", "CR": "Costa Rica", "US": "United States", "PA": "Panama",
    "PE": "Peru", "GL": "Greenland", "GB": "United Kingdom", "AU": "Australia", "CA": "Canada",
    "FK": "Falkland Islands", "CL": "Chile", "FO": "Faroe Islands", "NO": "Norway", "JP": "Japan",
    "BR": "Brazil", "SN": "Senegal", "NZ": "New Zealand", "AQ": "Antarctica", "MX": "Mexico",
    "PH": "Philippines", "ZA": "South Africa", "PT": "Portugal", "AO": "Angola", "ES": "Spain",
    "IT": "Italy", "EC": "Ecuador", "GS": "South Georgia", "NL": "Netherlands", "NC": "New Caledonia",
    "AT": "Austria", "MZ": "Mozambique", "XX": "Unknown", "MG": "Madagascar", "PK": "Pakistan",
    "TJ": "Tajikistan", "DE": "Germany", "RU": "Russia", "UZ": "Uzbekistan", "TD": "Chad",
    "TH": "Thailand", "LB": "Lebanon", "LA": "Laos", "SG": "Singapore", "CM": "Cameroon",
    "HK": "Hong Kong", "ID": "Indonesia", "AR": "Argentina", "BE": "Belgium", "UA": "Ukraine",
    "JE": "Jersey", "CD": "Democratic Republic of the Congo", "GA": "Gabon", "CO": "Colombia",
    "UY": "Uruguay", "CN": "China", "FJ": "Fiji", "SA": "Saudi Arabia", "BM": "Bermuda",
    "FM": "Micronesia", "KY": "Cayman Islands", "TR": "Turkey", "KI": "Kiribati", "TO": "Tonga",
    "AG": "Antigua and Barbuda", "SV": "El Salvador", "MV": "Maldives", "BZ": "Belize",
    "AE": "United Arab Emirates", "DZ": "Algeria", "IQ": "Iraq", "MD": "Moldova", "TL": "Timor-Leste",
    "MY": "Malaysia", "VN": "Vietnam", "KH": "Cambodia", "SH": "Saint Helena", "AL": "Albania",
    "DO": "Dominican Republic", "CU": "Cuba", "SC": "Seychelles", "SE": "Sweden", "LK": "Sri Lanka",
    "TZ": "Tanzania", "ZM": "Zambia", "CF": "Central African Republic", "LR": "Liberia",
    "ZW": "Zimbabwe", "MC": "Monaco", "KE": "Kenya", "TN": "Tunisia", "LV": "Latvia", "UG": "Uganda",
    "BW": "Botswana", "MN": "Mongolia", "KG": "Kyrgyzstan", "KZ": "Kazakhstan", "QA": "Qatar",
    "NE": "Niger", "BG": "Bulgaria", "HU": "Hungary", "CZ": "Czech Republic", "FI": "Finland",
    "GR": "Greece", "LT": "Lithuania", "LU": "Luxembourg", "MW": "Malawi", "SK": "Slovakia",
    "SZ": "Eswatini", "KR": "South Korea", "OM": "Oman", "RO": "Romania", "BD": "Bangladesh",
    "AM": "Armenia", "ML": "Mali", "BT": "Bhutan", "CG": "Congo", "CI": "Côte d'Ivoire", "GN": "Guinea",
    "SL": "Sierra Leone", "GW": "Guinea-Bissau", "TC": "Turks and Caicos Islands", "JM": "Jamaica",
    "BS": "Bahamas", "VG": "British Virgin Islands", "MM": "Myanmar", "PG": "Papua New Guinea",
    "LC": "Saint Lucia", "GT": "Guatemala", "CW": "Curacao", "BH": "Bahrain", "KW": "Kuwait",
    "BB": "Barbados", "HN": "Honduras", "SD": "Sudan", "CY": "Cyprus", "IR": "Iran", "LY": "Libya",
    "MA": "Morocco", "IL": "Israel", "EG": "Egypt", "NG": "Nigeria", "MR": "Mauritania", "KM": "Comoros",
    "GP": "Guadeloupe", "SB": "Solomon Islands", "IM": "Isle of Man", "LS": "Lesotho", "GH": "Ghana",
    "RS": "Serbia", "TW": "Taiwan", "PF": "French Polynesia", "PW": "Palau", "CK": "Cook Islands",
    "IE": "Ireland", "IN": "India", "JO": "Jordan", "MU": "Mauritius", "KN": "Saint Kitts and Nevis",
    "BQ": "Caribbean Netherlands", "BN": "Brunei", "TG": "Togo", "NI": "Nicaragua", "AD": "Andorra",
    "GE": "Georgia", "AW": "Aruba", "AZ": "Azerbaijan", "BA": "Bosnia and Herzegovina", "MO": "Macau",
    "BL": "Saint Barthelemy", "BO": "Bolivia", "BV": "Bouvet Island", "BY": "Belarus", "MT": "Malta",
    "PY": "Paraguay", "DM": "Dominica", "EE": "Estonia", "ER": "Eritrea", "ET": "Ethiopia",
    "GD": "Grenada", "GG": "Guernsey", "GI": "Gibraltar", "GQ": "Equatorial Guinea", "GU": "Guam",
    "GY": "Guyana", "HR": "Croatia", "HT": "Haiti", "LI": "Liechtenstein", "ME": "Montenegro",
    "MF": "Saint Martin", "MK": "North Macedonia", "NP": "Nepal", "RE": "Reunion", "SM": "San Marino",
    "SU": "Soviet Union", "SX": "Sint Maarten", "TF": "French Southern Territories", "TM": "Turkmenistan",
    "VA": "Vatican City", "VE": "Venezuela", "VI": "U.S. Virgin Islands", "KP": "North Korea",
    "VU": "Vanuatu", "YE": "Yemen", "SY": "Syria", "SO": "Somalia", "CV": "Cape Verde",
    "VC": "Saint Vincent and the Grenadines", "BF": "Burkina Faso", "RW": "Rwanda", "AI": "Anguilla",
    "MQ": "Martinique", "MS": "Montserrat", "BI": "Burundi", "MP": "Northern Mariana Islands", "WS": "Samoa",
    "TK": "Tokelau", "CC": "Cocos Islands", "AS": "American Samoa", "AF": "Afghanistan",
    "EH": "Western Sahara", "GF": "French Guiana", "GM": "Gambia", "MH": "Marshall Islands",
    "SJ": "Svalbard and Jan Mayen", "ST": "Sao Tome and Principe", "SS": "South Sudan",
    "PM": "Saint Pierre and Miquelon", "YT": "Mayotte", "NA": "Namibia",
    "AX": "Åland Islands", "BQ": "Bonaire, Saint Eustatius and Saba", "BV": "Bouvet Island",
    "CP": "Clipperton", "CX": "Christmas Island", "HM": "Heard and McDonald Islands",
    "HS": "High Seas", "NF": "Norfolk Island", "NR": "Nauru", "NU": "Niue", 
    "PN": "Pitcairn Islands", "PS": "State of Palestine", "TV": "Tuvalu",
    "UM": "United States Minor Outlying Islands", "WF": "Wallis and Futuna Islands",
    "XV": "Various", "AN": "Netherlands Antilles", "CS": "Former Serbia and Montenegro",
    "DD": "Former East Germany", "IO": "British Indian Ocean Territory", "NT": "Neutral Zone",
    "PC": "Former Pacific Trust Territory", "XA": "French Antilles", "XC": "Caribbean",
    "XE": "Europe", "XF": "Africa", "XM": "South America", "XS": "Asia", 
    "YU": "Former Yugoslavia", "ZC": "Former Czechoslovakia", "ZZ": "High seas"}
df_all["Importer_EN"] = df_all["Importer"].map(country_dict).fillna(df_all["Importer"])
df_all["Exporter_EN"] = df_all["Exporter"].map(country_dict).fillna(df_all["Exporter"])
df_all["Origin_EN"] = df_all["Origin"].map(country_dict).fillna(df_all["Origin"])

print(df_all[["Importer", "Importer_EN", "Exporter", "Exporter_EN", "Origin", "Origin_EN"]].head())


  Importer          Importer_EN Exporter  Exporter_EN Origin      Origin_EN
0       IS              Iceland       DK      Denmark     XX        Unknown
1       FR               France       CH  Switzerland     MO          Macau
2       FR               France       CH  Switzerland     XX        Unknown
3       TT  Trinidad and Tobago       SR     Suriname    NaN            NaN
4       CH          Switzerland       SI     Slovenia     US  United States


In [16]:
df_all["Importer_EN"] = df_all["Importer"].map(country_dict).fillna(df_all["Importer"])
df_all["Exporter_EN"] = df_all["Exporter"].map(country_dict).fillna(df_all["Exporter"])
df_all["Origin_EN"] = df_all["Origin"].map(country_dict).fillna(df_all["Origin"])

### Replacing Missing Origin_EN with Exporter_EN

In [17]:
    df_all["Origin_EN"] = df_all["Origin_EN"].fillna(df_all["Exporter_EN"])

## Converting Purpose Codes to English Descriptions

In [18]:
purpose_dict = {
    "B": "Breeding in captivity or artificial propagation",
    "E": "Educational",
    "G": "Botanical garden",
    "H": "Hunting trophy",
    "L": "Law enforcement/judicial/forensic",
    "M": "Medical (including biomedical research)",
    "N": "Reintroduction or introduction into the wild",
    "P": "Personal",
    "Q": "Circus or travelling exhibition",
    "S": "Scientific",
    "T": "Commercial",
    "Z": "Zoo"
}

df_all["Purpose_EN"] = df_all["Purpose"].map(purpose_dict).fillna(df_all["Purpose"])

print(df_all[["Purpose", "Purpose_EN"]].head(10))


  Purpose                       Purpose_EN
0       S                       Scientific
1       E                      Educational
2       Q  Circus or travelling exhibition
3       E                      Educational
4       T                       Commercial
5       S                       Scientific
6       S                       Scientific
7       S                       Scientific
8       S                       Scientific
9       S                       Scientific


## Converting Source Codes to English Descriptions

In [19]:
source_dict = {
    "A": "Artificially propagated plants",
    "C": "Captive-bred animals",
    "D": "Appendix-I captive-bred / commercial plants",
    "F": "Born in captivity (F1+)",
    "I": "Confiscated/seized",
    "O": "Pre-Convention specimens",
    "R": "Ranched specimens",
    "U": "unknown",
    "W": "Wild",
    "X": "Marine environment",
    "Y": "Assisted production plants"
}

df_all["Source_EN"] = df_all["Source"].map(source_dict).fillna(df_all["Source"])

print(df_all[["Source", "Source_EN"]].head(10))


  Source Source_EN
0      W      Wild
1      U   unknown
2      U   unknown
3      W      Wild
4      W      Wild
5      W      Wild
6      W      Wild
7      W      Wild
8      W      Wild
9      W      Wild


### Assigning Threat Status and Protection Level Based on CITES Appendix

## Explanation

### Appendices Overview

**Appendix I (I)**  
- Species facing the **highest risk of extinction**.  
- International trade is **generally prohibited**, except under exceptional circumstances.

**Appendix II (II)**  
- Species not necessarily threatened with extinction now but may become so **without controlled trade**.  
- Trade requires **regulation to avoid overexploitation**.

**Appendix III (III)**  
- Species protected in at least one country that has requested **assistance from other CITES parties** to control trade.  
- Trade is **allowed but monitored**.

### New Columns

- **Threat_Status**: A short, interpretable description of the species’ risk of extinction.  
- **Protection_Level**: A simplified level of legal protection for international trade.



In [20]:
app_dict_simple = {
    "I": {"Threat_Status": "Endangered", "Protection_Level": "High"},
    "II": {"Threat_Status": "Vulnerable", "Protection_Level": "Medium"},
    "III": {"Threat_Status": "Protected", "Protection_Level": "Low"}
}

df_all["Threat_Status"] = df_all["App."].map(lambda x: app_dict_simple.get(x, {}).get("Threat_Status"))
df_all["Protection_Level"] = df_all["App."].map(lambda x: app_dict_simple.get(x, {}).get("Protection_Level"))

print(df_all[["App.", "Threat_Status", "Protection_Level"]].head())


  App. Threat_Status Protection_Level
0    I    Endangered             High
1    I    Endangered             High
2    I    Endangered             High
3    I    Endangered             High
4    I    Endangered             High


In [21]:
missing_n_rows = df_all[df_all["App."] == "N"]
unique_missing_n_taxa = missing_n_rows["Taxon"].unique()
print(unique_missing_n_taxa)


['Physignathus cocincinus' 'Anas melleri' 'Pterapogon kauderni'
 'Elaphe carinata' 'Elaphe radiata' 'Elaphe taeniura' 'Enhydris bocourti'
 'Homalopsis buccata' 'Leioheterodon madagascariensis' 'Ptyas korros'
 'Columba livia' 'Gallicolumba criniger' 'Conraua goliath'
 'Lonchura punctulata' 'Rhacodactylus auriculatus'
 'Rhacodactylus ciliatus' 'Rhacodactylus leachianus'
 'Teratoscincus scincus' 'Zonosaurus quadrilineatus' 'Haliotis midae'
 'Lapemis curtus' 'Hystrix cristata' 'Ctenosaura quinquecarinata'
 'Dendrolagus goodfellowi' 'Graphium stresemanni' 'Potamotrygon falkneri'
 'Echinotriton andersoni' 'Tylototriton taliangensis'
 'Tribolonotus gracilis' 'Tribolonotus novaeguineae' 'Lamprotornis regius'
 'Struthio spp.' 'Carduelis carduelis' 'Agalychnis lemur'
 'Potamotrygon tigrina' 'Struthio camelus' 'Mergus octosetaceus'
 'Atelopus balios' 'Atelopus barbotini' 'Atelopus hoogmoedi'
 'Atelopus spumarius']


### Merging Trade Data with Full Species List

In [22]:
df_species = pd.read_csv("Index.csv", keep_default_na=False)

In [23]:
df_index = pd.read_csv("Index.csv")
df_index = df_index[['FullName', 'EnglishNames', 'CurrentListing']]
df_merged = df_all.merge(df_index, left_on="Taxon", right_on="FullName", how="left")
missing = df_merged[df_merged['EnglishNames'].isna()]['Taxon'].unique()
print("Missing matches:", missing)
df_merged.to_csv("trade_data_enriched.csv", index=False)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11192\383844461.py:1: DtypeWarning: Columns (2,3,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_index = pd.read_csv("Index.csv")


Missing matches: ['Balaenopteridae spp.' 'Boidae spp.' 'Capra falconeri heptneri' ...
 'Atelopus barbotini' 'Atelopus hoogmoedi' 'Atelopus spumarius']


In [24]:
index_df = pd.read_csv("Index.csv")

taxon_all = df_all["Taxon"].unique()
taxon_index = index_df["FullName"].unique()

missing_taxa = [t for t in taxon_all if t not in taxon_index]
print("Number of missing taxa:", len(missing_taxa))

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11192\517954032.py:1: DtypeWarning: Columns (2,3,29) have mixed types. Specify dtype option on import or set low_memory=False.
  index_df = pd.read_csv("Index.csv")


Number of missing taxa: 446


## Classifying Terms into Categories

In [25]:
import pandas as pd
import re

df = df_all.copy()

t = df["Term"].fillna("").astype(str).str.strip().str.upper()

live_keys = {"LIVE","EGG (LIVE)","FINGERLINGS","PUPAE"}

raw_keys = [
    "FEATHER","BALEEN","BONE PIECE","PIECE – BONE","PIECE - BONE","BONES","SKINS","SKIN PIECE","SKIN",
    "SKULL","SKELETON","BODIES","BODY","TEETH","TOOTH","SHELLS","SHELL","SCALES","SCALE","CARAPACE",
    "TUSKS","TUSK","IVORY PIECE","PIECE – IVORY","PIECE - IVORY","EGGS","EGG","HORNS","HORN","HORN PIECE",
    "RAW CORAL","CORAL (RAW)","SIDES","SIDE","TAILS","TAIL","FEET","FOOT","CLAWS","CLAW","GALL BLADDERS",
    "GALL BLADDER","PLATES","PLATE","EARS","EAR","TRUNK","GALL","GILL PLATES","GILL PLATE","FIN (DRIED)",
    "FIN (WET)","FIN","HAIR","SAWFISH ROSTRUM","GENITALIA","SWIM BLADDERS"
]

manuf_keys = [
    "LEATHER PRODUCTS (SMALL)","LEATHER PRODUCT (SMALL)","LPS",
    "LEATHER PRODUCTS (LARGE)","LEATHER PRODUCT (LARGE)","LPL",
    "BONE CARVINGS","BONE CARVING","TROPHIES","TROPHY","GARMENTS","GARMENT",
    "CLOTH","JEWELLERY - IVORY","JEWELRY - IVORY","IVORY CARVINGS","IVORY CARVING",
    "PIANO KEYS","HORN CARVINGS","HORN CARVING","CARVINGS","CARVING","RUG","COSMETICS",
    "FUR PRODUCTS (SMALL)","FUR PRODUCTS (LARGE)","HAIR PRODUCTS","JEWELLERY","JEWELRY"
]

food_keys = ["MEAT","CAVIAR","FROG LEGS"]

proc_keys = ["EXTRACT","POWDER","MUSK","OIL","MEDICINE","DERIVATIVES","FIBRES","FIBERS","PEARL","THREAD"]

def classify_term(x):
    u = re.sub(r"\s+", " ", x).strip().upper()
    if u in live_keys:
        return "Live Specimens"
    if u == "UNSPECIFIED" or u == "":
        return "Unspecified"
    for k in raw_keys:
        if k in u:
            return "Raw Body Parts"
    for k in manuf_keys:
        if k in u:
            return "Manufactured Products"
    for k in food_keys:
        if k in u:
            return "Food Products"
    for k in proc_keys:
        if k in u:
            return "Processed Materials"
    return "Unspecified"

df["Term_Group"] = t.map(classify_term)

counts = df["Term_Group"].value_counts(dropna=False)
perc = (counts / counts.sum() * 100).round(2)
summary = pd.DataFrame({"Count": counts, "Percent": perc}).reset_index().rename(columns={"index":"Term_Group"})
print(summary.to_string(index=False))

           Term_Group  Count  Percent
Manufactured Products  41371    36.67
       Live Specimens  37903    33.60
       Raw Body Parts  24276    21.52
          Unspecified   7801     6.92
        Food Products    882     0.78
  Processed Materials    573     0.51


In [26]:
df_all = df.copy()


In [27]:
u = df_all["App."].dropna().astype(str).str.strip()
u = u[u != ""]
uniq = sorted(u.unique(), key=str.upper)

print("Unique values in App.:")
for v in uniq:
    print(v)
print("Total:", len(uniq))


Unique values in App.:
I
II
III
N
Total: 4


### Correcting `App.` Values Using `CurrentListing`

In [28]:
mask_condition = (
    (df_merged['App.'] == 'N') & 
    df_merged['CurrentListing'].notna() & 
    (df_merged['CurrentListing'] != '') & 
    (df_merged['CurrentListing'] != 'I/NC')
)

count_before = mask_condition.sum()
print(f"Number of records to update: {count_before}")

df_merged.loc[mask_condition, 'App.'] = df_merged.loc[mask_condition, 'CurrentListing']

count_after = (df_merged['App.'] != 'N').sum()
print(f"Number of records with App. no longer 'N': {count_after}")

if count_before > 0:
    updated_sample = df_merged[mask_condition][['Taxon', 'App.', 'CurrentListing', 'Year']].head(10)
    print(updated_sample)


Number of records to update: 50
Number of records with App. no longer 'N': 112397
                            Taxon App. CurrentListing  Year
26633     Physignathus cocincinus   II             II  2019
26634     Physignathus cocincinus   II             II  2019
26635     Physignathus cocincinus   II             II  2019
26636     Physignathus cocincinus   II             II  2019
26637     Physignathus cocincinus   II             II  2019
26638     Physignathus cocincinus   II             II  2019
26639     Physignathus cocincinus   II             II  2019
26754  Ctenosaura quinquecarinata   II             II  2019
26755  Ctenosaura quinquecarinata   II             II  2019
26756  Ctenosaura quinquecarinata   II             II  2019


In [29]:
remaining_n_count = (df_merged['App.'] == 'N').sum()
total_records = len(df_merged)

print(f"Remaining records with App. = 'N': {remaining_n_count}")
print(f"Total records: {total_records}")
print(f"Percentage of remaining 'N': {(remaining_n_count/total_records)*100:.2f}%")

remaining_n_records = df_merged[df_merged['App.'] == 'N']
if len(remaining_n_records) > 0:
    print(f"\nCurrentListing values for remaining 'N' records:")
    print(remaining_n_records['CurrentListing'].value_counts())
    print(f"\nSample of remaining 'N' records:")
    print(remaining_n_records[['Taxon', 'App.', 'CurrentListing', 'Year']].head(10))

Remaining records with App. = 'N': 409
Total records: 112806
Percentage of remaining 'N': 0.36%

CurrentListing values for remaining 'N' records:
CurrentListing
I/NC    7
Name: count, dtype: int64

Sample of remaining 'N' records:
                     Taxon App. CurrentListing  Year
26640         Anas melleri    N            NaN  2019
26641         Anas melleri    N            NaN  2019
26642  Pterapogon kauderni    N            NaN  2019
26643  Pterapogon kauderni    N            NaN  2019
26644  Pterapogon kauderni    N            NaN  2019
26645  Pterapogon kauderni    N            NaN  2019
26646  Pterapogon kauderni    N            NaN  2019
26647  Pterapogon kauderni    N            NaN  2019
26648  Pterapogon kauderni    N            NaN  2019
26649  Pterapogon kauderni    N            NaN  2019


### Removing Rest Records with `App.` = "N"

In [30]:
records_to_delete = (df_merged['App.'] == 'N').sum()
total_before = len(df_merged)

print(f"Records to delete (App. = 'N'): {records_to_delete}")
print(f"Total records before deletion: {total_before}")

df_merged = df_merged[df_merged['App.'] != 'N']

total_after = len(df_merged)
print(f"Total records after deletion: {total_after}")
print(f"Records deleted: {total_before - total_after}")

remaining_n = (df_merged['App.'] == 'N').sum()
print(f"Remaining records with App. = 'N': {remaining_n}")

Records to delete (App. = 'N'): 409
Total records before deletion: 112806
Total records after deletion: 112397
Records deleted: 409
Remaining records with App. = 'N': 0


In [31]:
print(df_all.head())

   Year App.                 Taxon     Class          Order         Family  \
0  2019    I  Haliaeetus albicilla      Aves  Falconiformes   Accipitridae   
1  2019    I        Harpia harpyja      Aves  Falconiformes   Accipitridae   
2  2019    I        Harpia harpyja      Aves  Falconiformes   Accipitridae   
3  2019    I        Harpia harpyja      Aves  Falconiformes   Accipitridae   
4  2019    I    Alligator sinensis  Reptilia     Crocodylia  Alligatoridae   

        Genus Importer Exporter Origin  ...  Reported_Quantity  Class_EN  \
0  Haliaeetus       IS       DK     XX  ...              30.00     Birds   
1      Harpia       FR       CH     MO  ...               2.66     Birds   
2      Harpia       FR       CH     XX  ...               2.00     Birds   
3      Harpia       TT       SR    NaN  ...               1.00     Birds   
4   Alligator       CH       SI     US  ...              11.00  Reptiles   

           Importer_EN  Exporter_EN      Origin_EN  \
0              Icela

In [32]:
records_to_delete = (df_merged['App.'] == 'N').sum()
total_before = len(df_merged)

print(f"Records to delete (App. = 'N'): {records_to_delete}")
print(f"Total records before deletion: {total_before}")

df_merged = df_merged[df_merged['App.'] != 'N']

total_after = len(df_merged)
print(f"Total records after deletion: {total_after}")
print(f"Records deleted: {total_before - total_after}")

remaining_n = (df_merged['App.'] == 'N').sum()
print(f"Remaining records with App. = 'N': {remaining_n}")

Records to delete (App. = 'N'): 0
Total records before deletion: 112397
Total records after deletion: 112397
Records deleted: 0
Remaining records with App. = 'N': 0


In [33]:
df_merged["Term_Group"] = df["Term_Group"]

### Renaming or Correcting Species Names

In [34]:
reptiles_df = df_merged[df_merged['Class'] == 'Reptilia']

top_reptiles = reptiles_df.groupby(['Taxon', 'EnglishNames']).size().reset_index(name='Count')
top_reptiles = top_reptiles.sort_values('Count', ascending=False).head(5)

print("Top 5 Reptile Taxa:")
print(top_reptiles[['Taxon', 'EnglishNames', 'Count']])

Top 5 Reptile Taxa:
                          Taxon  \
5    Alligator mississippiensis   
377            Varanus salvator   
216    Malayopython reticulatus   
369           Varanus niloticus   
111        Crocodylus niloticus   

                                          EnglishNames  Count  
5                            American Alligator, Gator   9339  
377  Common Water Monitor, Malayan Monitor, No-mark...   5208  
216  Asian reticulated python, Java Rock Python, Re...   4472  
369           African Small-grain Lizard, Nile Monitor   2984  
111                  African Crocodile, Nile Crocodile   1677  


In [35]:
reptiles_df = df_merged[df_merged['Class'] == 'Reptilia']

top_reptiles = reptiles_df.groupby(['Taxon', 'EnglishNames']).size().reset_index(name='Count')
top_reptiles = top_reptiles.sort_values('Count', ascending=False).head(5)

name_mapping = {
    'Alligator mississippiensis': 'American alligator',
    'Varanus salvator': 'Asian water monitor', 
    'Malayopython reticulatus': 'Asian python',
    'Varanus niloticus': 'Nile monitor',
    'Crocodylus niloticus': 'Nile crocodile'
}

top_reptiles['EnglishName'] = top_reptiles['Taxon'].map(name_mapping)

print("Top 5 Reptile Taxa:")
print(top_reptiles[['Taxon', 'EnglishName', 'Count']].to_string(index=False))

Top 5 Reptile Taxa:
                     Taxon         EnglishName  Count
Alligator mississippiensis  American alligator   9339
          Varanus salvator Asian water monitor   5208
  Malayopython reticulatus        Asian python   4472
         Varanus niloticus        Nile monitor   2984
      Crocodylus niloticus      Nile crocodile   1677


In [36]:
mammals_df = df_merged[df_merged['Class'] == 'Mammalia']

top_mammals = mammals_df.groupby(['Taxon', 'EnglishNames']).size().reset_index(name='Count')
top_mammals = top_mammals.sort_values('Count', ascending=False).head(5)

print("Top 5 Mammal Taxa:")
print(top_mammals[['Taxon', 'EnglishNames', 'Count']].to_string(index=False))

Top 5 Mammal Taxa:
                 Taxon                                   EnglishNames  Count
       Vicugna vicugna                        Vicugna, Vicuna, Vicuña   2029
    Loxodonta africana    African Elephant, African Savannah Elephant   1684
      Ursus americanus                            American Black Bear   1452
Hippopotamus amphibius Common hippopotamus, Hippopotamus, Large Hippo   1417
Giraffa camelopardalis                                        Giraffe   1366


In [37]:
mammals_df = df_merged[df_merged['Class'] == 'Mammalia']

top_mammals = mammals_df.groupby(['Taxon', 'EnglishNames']).size().reset_index(name='Count')
top_mammals = top_mammals.sort_values('Count', ascending=False).head(5)

name_mapping = {
    'Vicugna vicugna': 'Lama vicugna',
    'Loxodonta africana': 'African Elephant',
    'Ursus americanus': 'American Black Bear',
    'Hippopotamus amphibius': 'Large Hippo',
    'Giraffa camelopardalis': 'Giraffe'
}

top_mammals['EnglishName'] = top_mammals['Taxon'].map(name_mapping)

print("Top 5 Mammal Taxa:")
print(top_mammals[['Taxon', 'EnglishName', 'Count']].to_string(index=False))

Top 5 Mammal Taxa:
                 Taxon         EnglishName  Count
       Vicugna vicugna        Lama vicugna   2029
    Loxodonta africana    African Elephant   1684
      Ursus americanus American Black Bear   1452
Hippopotamus amphibius         Large Hippo   1417
Giraffa camelopardalis             Giraffe   1366


In [38]:
corals_df = df_merged[df_merged['Class'] == 'Anthozoa']

top_corals = corals_df.groupby(['Taxon', 'EnglishNames']).size().reset_index(name='Count')
top_corals = top_corals.sort_values('Count', ascending=False).head(5)

print("Top 5 Coral Taxa:")
print(top_corals[['Taxon', 'EnglishNames', 'Count']].to_string(index=False))

Top 5 Coral Taxa:
                  Taxon                                               EnglishNames  Count
       Euphyllia ancora                                 Anchor Coral, Hammer Coral    568
Trachyphyllia geoffroyi Crater Coral, Folded Coral, Open brain coral, Puffed Coral    529
 Catalaphyllia jardinei                                              Elegant Coral    474
      Plerogyra sinuosa                                              Bladder Coral    432
       Euphyllia divisa                                            Frogspawn Coral    428


In [39]:
corals_df = df_merged[df_merged['Class'] == 'Anthozoa']

top_corals = corals_df.groupby(['Taxon', 'EnglishNames']).size().reset_index(name='Count')
top_corals = top_corals.sort_values('Count', ascending=False).head(5)

name_mapping = {
    'Euphyllia ancora': 'Hammer Coral',
    'Trachyphyllia geoffroyi': 'Puffed Coral',
    'Catalaphyllia jardinei': 'Elegant Coral',
    'Plerogyra sinuosa': 'Bladder Coral',
    'Euphyllia divisa': 'Frogspawn Coral'
}

top_corals['EnglishName'] = top_corals['Taxon'].map(name_mapping)

print("Top 5 Coral Taxa:")
print(top_corals[['Taxon', 'EnglishName', 'Count']].to_string(index=False))

Top 5 Coral Taxa:
                  Taxon     EnglishName  Count
       Euphyllia ancora    Hammer Coral    568
Trachyphyllia geoffroyi    Puffed Coral    529
 Catalaphyllia jardinei   Elegant Coral    474
      Plerogyra sinuosa   Bladder Coral    432
       Euphyllia divisa Frogspawn Coral    428


In [40]:
corals_df = df_merged[df_merged['Class'] == 'Anthozoa']

top_corals = corals_df.groupby(['Taxon', 'EnglishNames']).size().reset_index(name='Count')
top_corals = top_corals.sort_values('Count', ascending=False).head(5)

name_mapping = {
    'Euphyllia ancora': 'Hammer Coral',
    'Trachyphyllia geoffroyi': 'Puffed Coral',
    'Catalaphyllia jardinei': 'Elegant Coral',
    'Plerogyra sinuosa': 'Bladder Coral',
    'Euphyllia divisa': 'Frogspawn Coral'
}

top_corals['EnglishName'] = top_corals['Taxon'].map(name_mapping)

print("Top 5 Coral Taxa:")
print(top_corals[['Taxon', 'EnglishName', 'Count']].to_string(index=False))


Top 5 Coral Taxa:
                  Taxon     EnglishName  Count
       Euphyllia ancora    Hammer Coral    568
Trachyphyllia geoffroyi    Puffed Coral    529
 Catalaphyllia jardinei   Elegant Coral    474
      Plerogyra sinuosa   Bladder Coral    432
       Euphyllia divisa Frogspawn Coral    428


In [41]:
birds_df = df_merged[df_merged['Class'] == 'Aves']

top_birds = birds_df.groupby(['Taxon', 'EnglishNames']).size().reset_index(name='Count')
top_birds = top_birds.sort_values('Count', ascending=False).head(5)

print("Top 5 Bird Taxa:")
print(top_birds[['Taxon', 'EnglishNames', 'Count']].to_string(index=False))

Top 5 Bird Taxa:
                  Taxon                                                        EnglishNames  Count
    Psittacus erithacus                           African grey parrot, Grey Parrot, Jacquot    274
           Ara ararauna                          Blue-and-gold Macaw, Blue-and-yellow Macaw    234
       Ara chloropterus                             Green-winged Macaw, Red-and-green Macaw    205
   Amazona ochrocephala Yellow-crowned Amazon, Yellow-crowned Parrot, Yellow-fronted Amazon    173
Pionites melanocephalus                            Black-headed Caique, Black-headed Parrot    148


In [42]:
birds_df = df_merged[df_merged['Class'] == 'Aves']

top_birds = birds_df.groupby(['Taxon', 'EnglishNames']).size().reset_index(name='Count')
top_birds = top_birds.sort_values('Count', ascending=False).head(5)

name_mapping = {
    'Psittacus erithacus': 'African grey parrot',
    'Ara ararauna': 'Blue gold Macaw',
    'Ara chloropterus': 'Green winged Macaw',
    'Amazona ochrocephala': 'Yellow crowned Amazon',
    'Pionites melanocephalus': 'Black headed Parrot'
}

top_birds['EnglishName'] = top_birds['Taxon'].map(name_mapping)

print("Top 5 Bird Taxa:")
print(top_birds[['Taxon', 'EnglishName', 'Count']].to_string(index=False))

Top 5 Bird Taxa:
                  Taxon           EnglishName  Count
    Psittacus erithacus   African grey parrot    274
           Ara ararauna       Blue gold Macaw    234
       Ara chloropterus    Green winged Macaw    205
   Amazona ochrocephala Yellow crowned Amazon    173
Pionites melanocephalus   Black headed Parrot    148


In [43]:
sharks_rays_df = df_merged[df_merged['Class'] == 'Elasmobranchii']

top_sharks_rays = sharks_rays_df.groupby(['Taxon', 'EnglishNames']).size().reset_index(name='Count')
top_sharks_rays = top_sharks_rays.sort_values('Count', ascending=False).head(5)

print("Top 5 Shark and Ray Taxa:")
print(top_sharks_rays[['Taxon', 'EnglishNames', 'Count']].to_string(index=False))

Top 5 Shark and Ray Taxa:
                   Taxon                                                                                                                                                                   EnglishNames  Count
       Isurus oxyrinchus                                                                                                                                                            Mako, Shortfin mako    343
Carcharhinus falciformis                                                                                                                                                                    Silky shark    248
          Sphyrna lewini Bronze hammerhead shark, gebuchteter Hammerhai, Hammerhead, Hammerhead shark, Kidney-headed shark, Scalloped hammerhead, Scalloped hammerhead shark, Southern hammerhead shark    169
     Potamotrygon motoro                                                                                                                          

In [44]:
sharks_rays_df = df_merged[df_merged['Class'] == 'Elasmobranchii']

top_sharks_rays = sharks_rays_df.groupby(['Taxon', 'EnglishNames']).size().reset_index(name='Count')
top_sharks_rays = top_sharks_rays.sort_values('Count', ascending=False).head(5)

name_mapping = {
    'Isurus oxyrinchus': 'Mako Shark',
    'Carcharhinus falciformis': 'Silky shark',
    'Sphyrna lewini': 'Bronze hammerhead shark',
    'Potamotrygon motoro': 'River stingray',
    'Sphyrna zygaena': 'Smooth hammerhead shark'
}

top_sharks_rays['EnglishName'] = top_sharks_rays['Taxon'].map(name_mapping)

print("Top 5 Shark and Ray Taxa:")
print(top_sharks_rays[['Taxon', 'EnglishName', 'Count']].to_string(index=False))

Top 5 Shark and Ray Taxa:
                   Taxon             EnglishName  Count
       Isurus oxyrinchus              Mako Shark    343
Carcharhinus falciformis             Silky shark    248
          Sphyrna lewini Bronze hammerhead shark    169
     Potamotrygon motoro          River stingray    122
         Sphyrna zygaena Smooth hammerhead shark     92


In [45]:
unique_purposes = df_merged['Purpose_EN'].unique()

print("Unique values in Purpose_EN:")
print(unique_purposes)


Unique values in Purpose_EN:
['Scientific' 'Educational' 'Circus or travelling exhibition' 'Commercial'
 'Personal' 'Hunting trophy' 'Medical (including biomedical research)'
 'Zoo' 'Reintroduction or introduction into the wild' nan
 'Breeding in captivity or artificial propagation'
 'Law enforcement/judicial/forensic' 'Botanical garden']


### Standardizing Purpose Names

In [46]:
import pandas as pd
import numpy as np

purpose_mapping = {
    "Circus or travelling exhibition": "Circus /exhibition",
    "Reintroduction or introduction into the wild": "Reintroduction",
    "Breeding in captivity or artificial propagation": "Breeding in captivity",
    "Law enforcement/judicial/forensic": "Law enforcement",
    "Medical (including biomedical research)": "Medical",
    pd.NA: "Unspecified",
    float('nan'): "Unspecified"
}

df_merged['Purpose_EN'] = df_merged['Purpose_EN'].replace(purpose_mapping)
df_merged['Purpose_EN'] = df_merged['Purpose_EN'].str.strip()
unique_purposes = df_merged['Purpose_EN'].unique()
for purpose in unique_purposes:
    print(purpose)


Scientific
Educational
Circus /exhibition
Commercial
Personal
Hunting trophy
Medical
Zoo
Reintroduction
Unspecified
Breeding in captivity
Law enforcement
Botanical garden


In [47]:
name_mapping_reptiles = {
    'Alligator mississippiensis': 'American alligator',
    'Varanus salvator': 'Asian water monitor', 
    'Malayopython reticulatus': 'Asian python',
    'Varanus niloticus': 'Nile monitor',
    'Crocodylus niloticus': 'Nile crocodile'
}

name_mapping_mammals = {
    'Vicugna vicugna': 'Lama vicugna',
    'Loxodonta africana': 'African Elephant',
    'Ursus americanus': 'American Black Bear',
    'Hippopotamus amphibius': 'Large Hippo',
    'Giraffa camelopardalis': 'Giraffe'
}

name_mapping_corals = {
    'Euphyllia ancora': 'Hammer Coral',
    'Trachyphyllia geoffroyi': 'Puffed Coral',
    'Catalaphyllia jardinei': 'Elegant Coral',
    'Plerogyra sinuosa': 'Bladder Coral',
    'Euphyllia divisa': 'Frogspawn Coral'
}

name_mapping_birds = {
    'Psittacus erithacus': 'African grey parrot',
    'Ara ararauna': 'Blue gold Macaw',
    'Ara chloropterus': 'Green winged Macaw',
    'Amazona ochrocephala': 'Yellow crowned Amazon',
    'Pionites melanocephalus': 'Black headed Parrot'
}

name_mapping_sharks_rays = {
    'Isurus oxyrinchus': 'Mako Shark',
    'Carcharhinus falciformis': 'Silky shark',
    'Sphyrna lewini': 'Bronze hammerhead shark',
    'Potamotrygon motoro': 'River stingray',
    'Sphyrna zygaena': 'Smooth hammerhead shark'
}

df_merged.loc[df_merged['Class'] == 'Reptilia', 'EnglishNames'] = df_merged.loc[df_merged['Class'] == 'Reptilia', 'Taxon'].map(name_mapping_reptiles)
df_merged.loc[df_merged['Class'] == 'Mammalia', 'EnglishNames'] = df_merged.loc[df_merged['Class'] == 'Mammalia', 'Taxon'].map(name_mapping_mammals)
df_merged.loc[df_merged['Class'] == 'Anthozoa', 'EnglishNames'] = df_merged.loc[df_merged['Class'] == 'Anthozoa', 'Taxon'].map(name_mapping_corals)
df_merged.loc[df_merged['Class'] == 'Aves', 'EnglishNames'] = df_merged.loc[df_merged['Class'] == 'Aves', 'Taxon'].map(name_mapping_birds)
df_merged.loc[df_merged['Class'] == 'Elasmobranchii', 'EnglishNames'] = df_merged.loc[df_merged['Class'] == 'Elasmobranchii', 'Taxon'].map(name_mapping_sharks_rays)


In [48]:
classes_of_interest = ['Reptilia', 'Aves', 'Elasmobranchii', 'Anthozoa', 'Mammalia']

for cls in classes_of_interest:
    class_df = df_merged[df_merged['Class'] == cls]
    
    top_taxa = class_df.groupby(['Taxon', 'EnglishNames']).size().reset_index(name='Count')
    top_taxa = top_taxa.sort_values('Count', ascending=False).head(5)
    
    print(f"\nTop 5 Taxa for Class: {cls}")
    print(top_taxa[['Taxon', 'EnglishNames', 'Count']].to_string(index=False))



Top 5 Taxa for Class: Reptilia
                     Taxon        EnglishNames  Count
Alligator mississippiensis  American alligator   9339
          Varanus salvator Asian water monitor   5208
  Malayopython reticulatus        Asian python   4472
         Varanus niloticus        Nile monitor   2984
      Crocodylus niloticus      Nile crocodile   1677

Top 5 Taxa for Class: Aves
                  Taxon          EnglishNames  Count
    Psittacus erithacus   African grey parrot    274
           Ara ararauna       Blue gold Macaw    234
       Ara chloropterus    Green winged Macaw    205
   Amazona ochrocephala Yellow crowned Amazon    173
Pionites melanocephalus   Black headed Parrot    148

Top 5 Taxa for Class: Elasmobranchii
                   Taxon            EnglishNames  Count
       Isurus oxyrinchus              Mako Shark    343
Carcharhinus falciformis             Silky shark    248
          Sphyrna lewini Bronze hammerhead shark    169
     Potamotrygon motoro          Ri

In [49]:
name_mapping = {
    'Scleractinia spp.': 'Stony corals',
    'Goniopora spp.': 'Flowerpot coral',
    'Acropora spp.': 'Table coral',
    'Euphyllia glabrescens': 'Torch coral',
    'Fungia spp.': 'Mushroom coral'
}

df_merged['EnglishNames'] = df_merged['Taxon'].map(name_mapping).fillna(df_merged['EnglishNames'])


### Counting Missing Threat and Protection Data

In [50]:
app_dict_simple = {
    "I": {"Threat_Status": "Endangered", "Protection_Level": "High"},
    "II": {"Threat_Status": "Vulnerable", "Protection_Level": "Medium"},
    "III": {"Threat_Status": "Protected", "Protection_Level": "Low"}
}

def fill_missing(row):
    if pd.isna(row['Threat_Status']) and pd.isna(row['Protection_Level']):
        app = row['App.']
        if app in app_dict_simple:
            row['Threat_Status'] = app_dict_simple[app]['Threat_Status']
            row['Protection_Level'] = app_dict_simple[app]['Protection_Level']
    return row

df_merged = df_merged.apply(fill_missing, axis=1)


In [51]:
missing_count = df_merged[
    df_merged['Threat_Status'].isna() & df_merged['Protection_Level'].isna()
].shape[0]

print(missing_count)


0


In [52]:
df_merged.to_csv("CITES WILDLIFE TRADE2.csv", index=False)


In [53]:
print(df_merged.head())

   Year App.                 Taxon     Class          Order         Family  \
0  2019    I  Haliaeetus albicilla      Aves  Falconiformes   Accipitridae   
1  2019    I        Harpia harpyja      Aves  Falconiformes   Accipitridae   
2  2019    I        Harpia harpyja      Aves  Falconiformes   Accipitridae   
3  2019    I        Harpia harpyja      Aves  Falconiformes   Accipitridae   
4  2019    I    Alligator sinensis  Reptilia     Crocodylia  Alligatoridae   

        Genus Importer Exporter Origin  ...  Exporter_EN      Origin_EN  \
0  Haliaeetus       IS       DK     XX  ...      Denmark        Unknown   
1      Harpia       FR       CH     MO  ...  Switzerland          Macau   
2      Harpia       FR       CH     XX  ...  Switzerland        Unknown   
3      Harpia       TT       SR    NaN  ...     Suriname       Suriname   
4   Alligator       CH       SI     US  ...     Slovenia  United States   

           Purpose_EN Source_EN Threat_Status Protection_Level  \
0          Sci

In [54]:
import pandas as pd, numpy as np, re

df = df_merged.copy() 

def norm_app(a):
    s = str(a).upper()
    s = s.replace("APPENDIX","").replace("ANNEX","").replace("APP.","").replace("APP","").replace(".","").strip()
    if re.fullmatch(r"I{3}", s): return "III"
    if re.fullmatch(r"II", s): return "II"
    if re.fullmatch(r"I", s): return "I"
    return "UNK"

def norm_purpose(p):
    x = str(p).strip().upper()
    m = {
        "T":"T","COMMERCIAL":"T",
        "P":"P","PERSONAL":"P",
        "Q":"Q","ENTERTAINMENT":"Q",
        "H":"H","HUNTING TROPHY":"H",
        "B":"B","BREEDING IN CAPTIVITY OR ARTIFICIAL PROPAGATION":"B",
        "M":"M","MEDICAL (INCLUDING BIOMEDICAL RESEARCH)":"M","MEDICAL":"M",
        "Z":"Z","ZOO":"Z",
        "G":"G","BOTANICAL GARDEN":"G",
        "S":"S","SCIENTIFIC":"S",
        "N":"N","REINTRODUCTION OR INTRODUCTION INTO THE WILD":"N",
        "E":"E","EDUCATIONAL":"E",
        "L":"L","LAW ENFORCEMENT/JUDICIAL/FORENSIC":"L",
        "":""
    }
    return m.get(x, x[:1] if x else "")

app = df.get("App.", "").map(norm_app)
purp = df.get("Purpose", "").map(norm_purpose)
term = df.get("Term", "").fillna("").astype(str).str.strip()
tax  = df.get("Taxon","").fillna("").astype(str).str.strip()
exp  = df.get("Exporter","").fillna("").astype(str).str.strip().str.upper()

term_lc = term.str.lower()
tax_lc  = tax.str.lower()

sens_always = term_lc.str.contains(r"\b(ivory|tusk|piano keys|jewell?ery\s*-\s*ivory|ivory carv(ing|ings)?|ivory piece(s)?|horns?|horn piece(s)?|horn carv(ing|ings)?|troph(y|ies)|sawfish rostrum)\b")
elephant_like = tax_lc.str.contains(r"\belephantidae\b|\bloxodonta\b|\belephas\b")
walrus_like   = tax_lc.str.contains(r"\bodoben(us|us)\s+rosmarus\b")
hippo_like    = tax_lc.str.contains(r"\bhippopotamus\s+amphibius\b")
whale_like    = tax_lc.str.contains(r"\bbalaenoptera\b")
polar_like    = tax_lc.str.contains(r"\bursus\s+maritimus\b")

jewellery_only = term_lc.str.fullmatch(r"\s*jewell?ery\s*")
hard_parts = term_lc.str.contains(r"\b(teeth?|skulls?|bones?|bone pieces?|bone carv(ing|ings)?)\b")

sens_cond = (jewellery_only & (elephant_like | walrus_like | hippo_like)) | (hard_parts & (elephant_like | walrus_like | hippo_like | whale_like | polar_like))

textile_terms = term_lc.str.contains(r"\b(cloth|garment|fibre|fiber|yarn|textile|fabric)\b")
is_live = term_lc.str.fullmatch(r"\s*live\s*")
vicuna_exception = tax_lc.str.fullmatch(r"\svicugna\s+vicugna\s*") & exp.eq("PE") & textile_terms & ~is_live

df["SensitivityFlag"] = np.where(vicuna_exception, "Not Sensitive", np.where(sens_always | sens_cond, "Sensitive", "Not Sensitive"))

bucket = np.select(
    [
        purp.isin(["T","P","Q"]),
        purp.isin(["H","B"]),
        purp.eq("M"),
        purp.isin(["Z","G"]),
        purp.isin(["S","N","E","L"]),
        purp.eq("")
    ],
    ["TPQ","HB","M","ZG","SNEL","UNSP"],
    default="OTHER"
)

base_map = {
    "TPQ": {"I":"Critical","II":"Medium","III":"Low"},
    "HB":  {"I":"Critical","II":"High","III":"Low"},
    "M":   {"I":"High","II":"Medium","III":"Low"},
    "ZG":  {"I":"Medium","II":"Low","III":"Low"},
    "SNEL":{"I":"Low","II":"Low","III":"Low"},
    "UNSP":{"I":"Critical","II":"Critical","III":"High"}
}

def base_level(b,a):
    if a not in {"I","II","III"}: return "Medium"
    d = base_map.get(b, None)
    if d is None: return "Medium"
    return d.get(a, "Medium")

df["EnforcementWeaknessBase"] = [base_level(b,a) for b,a in zip(bucket, app)]

order = ["Low","Medium","High","Critical"]
up1 = {"Low":"Medium","Medium":"High","High":"Critical","Critical":"Critical"}

def escalate(level, appendix, sensitive):
    if sensitive != "Sensitive": return level
    if appendix == "I": return "Critical"
    if appendix == "II": return up1.get(level, level)
    if appendix == "III": return up1.get(level, level)
    return level

df["EnforcementWeaknessFinal"] = [escalate(l,a,s) for l,a,s in zip(df["EnforcementWeaknessBase"], app, df["SensitivityFlag"])]

summary = df["EnforcementWeaknessFinal"].value_counts().reindex(order, fill_value=0)
perc = (summary/summary.sum()*100).round(2)
print("Enforcement Weakness (final):")
for k in order: print(f"- {k}: {int(summary[k])} ({perc[k]}%)")
print(f"Total: {int(summary.sum())}")

by_app = df.groupby(app.rename("Appendix"))["EnforcementWeaknessFinal"].value_counts().unstack(fill_value=0).reindex(columns=order, fill_value=0)
by_app["Total"] = by_app.sum(axis=1)
print("\nBy Appendix:")
print(by_app.to_string())


df_merged = df.copy()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11192\943478167.py:41: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sens_always = term_lc.str.contains(r"\b(ivory|tusk|piano keys|jewell?ery\s*-\s*ivory|ivory carv(ing|ings)?|ivory piece(s)?|horns?|horn piece(s)?|horn carv(ing|ings)?|troph(y|ies)|sawfish rostrum)\b")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11192\943478167.py:43: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  walrus_like   = tax_lc.str.contains(r"\bodoben(us|us)\s+rosmarus\b")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11192\943478167.py:49: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  hard_parts = term_lc.str.contains(r"\b(teeth?|skulls?|bones?|bone pieces?|bone carv(ing|ings)?)\b")
C:\Users\Lenovo\AppDat

Enforcement Weakness (final):
- Low: 13049 (11.61%)
- Medium: 83689 (74.46%)
- High: 5960 (5.3%)
- Critical: 9699 (8.63%)
Total: 112397

By Appendix:
EnforcementWeaknessFinal   Low  Medium  High  Critical   Total
Appendix                                                      
I                         1875      75    18      2334    4302
II                        8529   81902  5942      7365  103738
III                       2645    1712     0         0    4357


In [55]:
df_merged.to_csv("trade_data_cleaned.csv", index=False)
